In [ ]:
from dataclasses import dataclass

import torch
from datasets import Dataset
from transformers import (
    LlamaForSequenceClassification,
    AutoTokenizer,
    EvalPrediction,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    BitsAndBytesConfig,
    PreTrainedTokenizerBase
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from sklearn.metrics import log_loss, accuracy_score, roc_auc_score

import warnings
warnings.filterwarnings('ignore')

In [ ]:
@dataclass
class Config:
    output_dir: str = 'output'
    checkpoint: str = "kakaocorp/kanana-1.5-2.1b-instruct-2505"  # 4-bit quantized gemma-2-9b-instruct
    max_length: int = 4096
    n_splits: int = 5
    fold_idx: int = 0
    optim_type: str = "adamw_torch"
    per_device_train_batch_size: int = 2
    gradient_accumulation_steps: int = 4  # global batch size is 16
    per_device_eval_batch_size: int = 8
    n_epochs: int = 3
    freeze_layers: int = 0  # there're 42 layers in total, we don't add adapters to the first 16 layers
    lr: float = 1e-5
    warmup_steps: int = 20
    lora_r: int = 64
    lora_alpha: float = 16
    lora_dropout: float = 0.1
    lora_bias: str = "none"

config = Config()

In [ ]:
training_args = TrainingArguments(
    output_dir=config.output_dir,
    do_eval=True,
    overwrite_output_dir=True,
    report_to="none",
    num_train_epochs=config.n_epochs,
    per_device_train_batch_size=config.per_device_train_batch_size,
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    per_device_eval_batch_size=config.per_device_eval_batch_size,
    logging_steps=100,
    eval_strategy="epoch",  # eval_strategy -> evaluation_strategy (올바른 키)
    save_strategy="steps",
    save_total_limit=3,
    save_steps=200,
    optim=config.optim_type,
    learning_rate=config.lr,
    weight_decay=0.01,  # ✅ weight decay 추가
    warmup_steps=config.warmup_steps,
    fp16=True,
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(config.checkpoint)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'
tokenizer.truncation_side='right'
tokenizer.add_eos_token = True

In [ ]:
from transformers import BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,                    # ✅ 8bit quantization 사용
    llm_int8_threshold=6.0,               # 기본값 (optional, 안정성 설정)
    llm_int8_has_fp16_weight=True,        # fp16 weight 유지 → mixed precision 가능
    llm_int8_skip_modules=["score"]       # quantization 제외할 모듈
)

In [ ]:
lora_config = LoraConfig(
    r=config.lora_r,
    lora_alpha=config.lora_alpha,
    target_modules=["q_proj", "k_proj", "v_proj","o_proj"],
    lora_dropout=config.lora_dropout,
    bias=config.lora_bias,
    task_type=TaskType.SEQ_CLS,
    modules_to_save=["score"]
)

In [ ]:
model = LlamaForSequenceClassification.from_pretrained(
    config.checkpoint,
    num_labels=1,
    #quantization_config=bnb_config,
    device_map="auto",
)

model.config.use_cache = False
#model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

In [ ]:
import pandas as pd
ds = pd.read_csv("train.csv")
ds = Dataset.from_pandas(ds)


In [ ]:
import re
class CustomTokenizer:
    def __init__(
        self,
        tokenizer: PreTrainedTokenizerBase,
        max_length: int
    ) -> None:
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def tmp(self, text):
        return re.sub(r'\s+', ' ', text)
        
    def __call__(self, batch: dict) -> dict:
        title = ["<Title>: " + t for t in batch["title"]]
        full = ["\n\n<Full text>: " + self.tmp(t) for t in batch["full_text"]]
        texts = [t + f for t, f in zip(title, full)]
        tokenized = self.tokenizer(texts, max_length=self.max_length, truncation=True)
        labels = [float(label) for label in batch["generated"]]
        
        return {**tokenized, "labels": labels}

In [ ]:
encode = CustomTokenizer(tokenizer, max_length=config.max_length)
ds = ds.map(encode, batched=True)


In [ ]:
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score
import torch
from transformers import EvalPrediction

def compute_metrics(eval_preds: EvalPrediction) -> dict:
    preds = eval_preds.predictions  # shape: (batch_size, 1)
    labels = eval_preds.label_ids   # shape: (batch_size,)

    # Apply sigmoid to get probabilities
    probs = torch.from_numpy(preds).float().sigmoid().numpy().squeeze()

    # Convert probabilities to binary predictions (threshold = 0.5)
    binary_preds = (probs >= 0.5).astype(int)

    # Compute metrics
    loss = log_loss(y_true=labels, y_pred=probs)
    auc = roc_auc_score(y_true=labels, y_score=probs)
    acc = accuracy_score(y_true=labels, y_pred=binary_preds)

    return {"auc": auc, "log_loss": loss, "accuracy": acc}


In [ ]:
def get_length_based_split_indices(dataset, max_len=512):
    train_idx = []
    eval_idx = []

    for i, input_ids in enumerate(dataset['input_ids']):
        if len(input_ids) < max_len:
            train_idx.append(i)
        else:
            eval_idx.append(i)

    return train_idx, eval_idx

# 사용 예시
train_idx, eval_idx = get_length_based_split_indices(ds, max_len=4096)


In [ ]:
from transformers import Trainer
import torch
import torch.nn as nn

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        """
        Custom loss computation. Receives `inputs` dictionary and returns the loss.
        You can add custom logic here.
        """
        outputs = model(**inputs)
        logits = outputs.get("logits")  # 또는 outputs.logits
        labels = inputs.get("labels")

        # 원하는 loss function 사용 예시
        loss_fct = nn.BCEWithLogitsLoss()
        loss = loss_fct(logits.view(-1), labels.view(-1))

        return (loss, outputs) if return_outputs else loss


In [ ]:

trainer = CustomTrainer(
    args=training_args, 
    model=model,
    tokenizer=tokenizer,
    train_dataset=ds.select(train_idx),
    eval_dataset=ds.select(eval_idx),
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
)
trainer.train()

In [ ]:
merged_model = model.merge_and_unload()
merged_model.save_pretrained("all")


In [ ]:
tokenizer.save_pretrained("all")

In [ ]:
import re
from typing import List, Dict
from transformers import PreTrainedTokenizerBase, AutoTokenizer

class CustomTokenizer:
    def __init__(
        self,
        tokenizer: PreTrainedTokenizerBase,
        max_length: int
    ) -> None:
        self.tokenizer = tokenizer
        self.max_length = max_length

    def _split_into_chunks(self, text: str, prefix: str) -> List[str]:
        """
        텍스트를 문장 단위로 분리하고, max_length에 맞춰 여러 청크로 나눕니다.
        각 청크는 title_prefix와 full_text_prefix를 포함하여 시작합니다.
        """
        sentences = re.split(r'(?<=[.?!])\s+', text)

        
        # [CLS], [SEP] 토큰 등을 고려하여 실제 텍스트에 사용할 수 있는 최대 토큰 수를 계산
        # 일반적으로 max_length - 2 (CLS, SEP) 이지만, 사용하는 모델의 토크나이저 특성에 따라 다를 수 있습니다.
        # 여기서는 간단히 max_length - 2로 가정합니다.
        effective_max_length = self.max_length - 2

        chunks = []
        current_chunk_sentences = []
        current_chunk_token_length = 0

        # 첫 번째 청크의 접두사 길이 계산 (예: "<Title>: " + "\n\n<Full text>: ")
        # 이 길이는 각 청크의 시작 부분에 포함될 것이므로, effective_max_length에서 제외해야 합니다.
        # 실제 청크에 들어갈 내용의 토큰 길이만 고려하기 위함입니다.
        initial_prefix_tokens = self.tokenizer.encode(prefix, add_special_tokens=False)
        prefix_token_length = len(initial_prefix_tokens)

        for i, sentence in enumerate(sentences):
            sentence_tokens = self.tokenizer.encode(sentence, add_special_tokens=False)
            
            # 새 청크의 시작이라면 접두사 길이를 포함하여 계산
            # 그렇지 않으면 문장 사이의 공백 토큰을 고려 (+1 또는 +a)
            len_to_add = len(sentence_tokens)
            if not current_chunk_sentences: # 새 청크의 첫 문장
                len_to_add += prefix_token_length
            else: # 기존 청크에 문장 추가
                len_to_add += len(self.tokenizer.encode(" ", add_special_tokens=False)) # 공백 토큰 길이

            if current_chunk_token_length + len_to_add > effective_max_length:
                # 현재까지 모아둔 문장들로 청크를 만듭니다.
                chunk_body = " ".join(current_chunk_sentences).strip()
                # 새 청크에는 반드시 title_prefix와 full_text_prefix를 붙입니다.
                chunks.append(prefix + chunk_body)
                
                # 새로운 청크를 현재 문장으로 시작
                current_chunk_sentences = [sentence]
                current_chunk_token_length = prefix_token_length + len(sentence_tokens)
            else:
                # 초과하지 않으면 현재 청크에 문장 추가
                current_chunk_sentences.append(sentence)
                current_chunk_token_length += len_to_add

        # 마지막 남은 문장들도 처리
        if current_chunk_sentences:
            chunk_body = " ".join(current_chunk_sentences).strip()
            chunks.append(prefix + chunk_body)
        
        return chunks

    def __call__(self, batch: dict) -> Dict[str, List]:
        # 각 원본 텍스트(제목 + 본문)에 대해 여러 청크를 생성합니다.
        all_tokenized_input_ids = []
        all_attention_mask = []
        all_token_type_ids = [] # BERT 계열에서 사용될 수 있음
        all_labels = []

        # 배치 내 각 샘플을 개별적으로 처리
        for i in range(len(batch["title"])):

            # _split_into_chunks를 호출하여 여러 청크를 얻습니다.
            # 이 청크들은 이미 title_prefix와 full_text_prefix가 붙은 상태입니다.
            processed_chunks = self._split_into_chunks(batch["full_text"][i], "<Title>: " + batch["title"][i] + "\n\n<Full text>: ")

            # 각 청크를 토큰화합니다.
            # labels는 원본 샘플에 대해 하나만 존재한다고 가정하고, 첫 번째 청크에만 연결합니다.
            # 만약 모든 청크에 동일한 라벨을 붙이고 싶다면, all_labels.extend([float(batch["generated"][i])] * len(processed_chunks)) 로 변경
            
            # 훈련 시 첫 번째 청크만 사용하거나, 모든 청크를 개별 샘플로 취급할 수 있습니다.
            # 여기서는 편의상 각 원본 샘플이 여러 청크를 가질 수 있도록 처리합니다.
            
            # 각 원본 데이터 포인트(batch["title"][i] + batch["full_text"][i])에 대해
            # 생성된 모든 청크를 최종 결과에 추가합니다.
            
            # 주의: 만약 'labels'가 각 원본 데이터 포인트(combined_text)에 대한 것이고,
            # 모델이 각 '청크'를 별도의 학습 샘플로 사용한다면,
            # 'labels' 리스트도 그에 맞춰 복제되거나 할당되어야 합니다.
            # 여기서는 하나의 원본 데이터에 대해 여러 청크가 생성되고,
            # 해당 원본 데이터의 labels를 각 청크에 반복하여 할당하는 방식으로 구현합니다.
            
            for chunk in processed_chunks:
                tokenized_chunk = self.tokenizer(
                    chunk,
                    max_length=self.max_length,
                    truncation=True, # 안전 장치
                    return_tensors="pt" # PyTorch 텐서로 반환 (선택 사항, 데이터셋 구성에 따라 다름)
                )
                all_tokenized_input_ids.append(tokenized_chunk['input_ids'].squeeze().tolist())
                all_attention_mask.append(tokenized_chunk['attention_mask'].squeeze().tolist())
                if 'token_type_ids' in tokenized_chunk: # BERT 계열에서만 존재
                    all_token_type_ids.append(tokenized_chunk['token_type_ids'].squeeze().tolist())
                
                # 원본 라벨을 현재 청크에 할당
                all_labels.append(float(batch["generated"][i]))
        
        result = {
            "input_ids": all_tokenized_input_ids,
            "attention_mask": all_attention_mask,
            "labels": all_labels
        }
        if all_token_type_ids:
            result["token_type_ids"] = all_token_type_ids
            
        return result

bert_tokenizer = AutoTokenizer.from_pretrained("team-lucid/deberta-v3-base-korean")
bert_tokenizer.pad_token = tokenizer.eos_token
bert_tokenizer.padding_side = 'right'
bert_tokenizer.truncation_side='right'
bert_tokenizer.add_eos_token = True
custom_tokenizer = CustomTokenizer(tokenizer=bert_tokenizer, max_length=512) # 짧은 길이로 테스트

ds = Dataset.from_csv("train.csv")
ds = ds.map(custom_tokenizer, batched=True, remove_columns=ds.column_names)


In [ ]:
class CustomTokenizer:
    def __init__(
        self,
        tokenizer: PreTrainedTokenizerBase,
        bert_tokenizer: PreTrainedTokenizerBase,
        max_length: int
    ) -> None:
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.bert_tokenizer = bert_tokenizer

    def __call__(self, batch: dict) -> dict:
        texts = [self.bert_tokenizer.decode(t, skip_special_tokens=True) for t in batch['input_ids']]

        tokenized = self.tokenizer(texts)
        
        # 기존 input_ids도 보존
        tokenized['original_input_ids'] = batch['input_ids']
        tokenized['original_attention_mask'] = batch['attention_mask']  # 선택적으로 추가

        # 기존 labels, token_type_ids도 보존하고 싶으면 아래처럼 추가
        if 'labels' in batch:
            tokenized['labels'] = batch['labels']
        if 'token_type_ids' in batch:
            tokenized['original_token_type_ids'] = batch['token_type_ids']

        return tokenized

In [ ]:
encode = CustomTokenizer(tokenizer, max_length=1024, bert_tokenizer=bert_tokenizer)

ds = ds.map(encode, batched=True, remove_columns=ds.column_names)


In [ ]:
from tqdm.notebook import tqdm
from transformers.data.data_collator import pad_without_fast_tokenizer_warning


@torch.no_grad()
@torch.cuda.amp.autocast()
def inference(ds, model, batch_size=8):
    preds = []
    model.eval()
    
    for start_idx in tqdm(range(0, len(ds), batch_size)):
        end_idx = min(start_idx + batch_size, len(ds))
        tmp = ds[start_idx:end_idx]
        input_ids = tmp["input_ids"]
        attention_mask = tmp["attention_mask"]
        inputs = pad_without_fast_tokenizer_warning(
            tokenizer,
            {"input_ids": input_ids, "attention_mask": attention_mask},
            padding="longest",
            pad_to_multiple_of=None,
            return_tensors="pt",
        )
        outputs = model(**inputs.to("cuda:0"))
        proba = outputs.logits.cpu()
        
        preds.extend(proba[:, 0].tolist())
    
    return preds

In [ ]:
a = inference(ds, model)

In [ ]:
all_ds = ds.add_column("generated", torch.sigmoid(torch.tensor(a)).tolist())

In [ ]:
all_ds

In [ ]:
all_ds = all_ds.remove_columns(['input_ids', 'attention_mask'])

In [ ]:
all_ds.save_to_disk("./stage1_llm")